## Get the fields used by all OCDS publications in the Registry

Use this notebook to get the list of the fields implemented by all the publishers in the Data Registry, for example, to check what publishers are publishing specific fields.

In [ ]:
# @title Get all the publications from the registry { display-mode: "form" }

publications = get_publications()

### Download all the publications, using the latest file available

In [ ]:
today = datetime.now(tz=timezone.utc)
results = []
for publication in publications:
    if publication["date_to"]:
        year = publication["date_to"][:4]
        if int(year) > today.year:
            year = today.year
    else:
        year = "full"
    download_file(publication, year)

### Extract the list of fields using cardinal

In [ ]:
final_dataset = pd.DataFrame()

for file in os.listdir("."):
    if file.endswith(".jsonl"):
        publisher = file.replace(".jsonl", "")
        coverage = !./ocdscardinal coverage  $file
        data = (
            pd.DataFrame.from_dict(json.loads(coverage[0]), orient="index", columns=["count"])
            .reset_index()
            .rename(columns={"index": "path"})
        )
        data["publisher"] = publisher
        final_dataset = pd.concat([final_dataset, data])

In [ ]:
final_dataset

Export the results as CSV

In [ ]:
final_dataset.to_csv("ocds_fields_from_all_publishers.csv", index=False)